In [1]:
import pandas as pd
import numpy as np
import random
from quantnote import BacktestEngine, Strategy

In [2]:
db_path = '/Users/j.nahmgoong/Documents/git/DB/MainDB.db'

engine = BacktestEngine(db_path)

Loading DB...
DB loaded in 44.61 seconds
[initialize] is executed in 50.96 seconds


In [3]:
engine.show_sample_strategy()

class myStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def compute_target(self, universe_list):
        target_weight = { }
        for ticker in universe_list:
            target_weight[ticker] = 1
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            assert False



In [4]:
class myStrategy(Strategy):
    def __init__(self, config):
        super().__init__()
        self.config = config
        
        print(self.config)

    def compute_target(self, universe_list):
        target_weight = { }
        
        input1 = self.config['input1']
        input2 = self.config['input2']
        unary = self.config['unary']
        binary = self.config['binary']
        unary_out = self.config['unary_out']
        
        factor_series_1 = self.compute_factor_series(universe_list, input1, type=unary)
        factor_series_2 = self.compute_factor_series(universe_list, input2, type=unary)
        
        if binary == 'add':
            factor_series = factor_series_1 + factor_series_2
        elif binary == 'sub':
            factor_series = factor_series_1 - factor_series_2
        elif binary == 'mul':
            factor_series = factor_series_1 * factor_series_2
        elif binary == 'div':
            factor_series = factor_series_1 / factor_series_2
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        assert len(factor_series) > 0.5*len(universe_list)
        
        if unary_out == 'raw':
            pass
        elif unary_out == 'rank':
            factor_series = factor_series.rank(ascending=False)
        elif unary_out == 'zscore':
            factor_series = (factor_series-factor_series.mean())/factor_series.std()
        
        factor_series = factor_series.dropna().sort_values(ascending=False)
        top = int(len(factor_series.index)/5)
        
        for ticker in factor_series.index:
            weight = factor_series[ticker]
            if weight > 0:
                target_weight[ticker] = weight
            
        target_weight = self.normalize(target_weight)
        return target_weight

    def custom_factor(self, ticker, ftype):
        if ftype == 'marketcap':
            return self.get_value('metric',ticker,'marketcap')
        else:
            return self.get_value('fundamentals',ticker,ftype)



In [ ]:
input_list = list(engine.cache['fundamentals']['AAPL'].columns[5:])
unary_list = ['raw', 'rank', 'zscore']
binary_list = ['add', 'sub', 'mul', 'div']

results = []

config_set = []

idx = 1
while True:
    x1, x2 = random.sample(input_list,2)
    unary = random.sample(unary_list,1)[0]
    binary = random.sample(binary_list,1)[0]
    unary_out = random.sample(unary_list,1)[0]
    
    config = {
    'input1':x1,
    'input2':x2,
    'unary':unary,
    'binary':binary,
    'unary_out':unary_out}
    
    if config in config_set:
        pass
    else:
        config_set.append(config)
    
    print('\n')
    print('Computing {}th factor'.format(idx))
    mys = myStrategy(config)
    try:
        engine.run_backtest(mys, '2016-01-01','2021-08-31',period='Q')
        engine.stat(bench=None)
        print(engine.stat_dic)

        results.append([engine.stat_dic['sharpe'],engine.stat_dic['cagr'],config])
        idx += 1
    except:
        print('Passed!')
        
    results.sort(key = lambda x:-x[0])
    print('Current best sharpe')
    print(results[0])




Computing 1th factor
{'input1': 'assetsnc', 'input2': 'price', 'unary': 'raw', 'binary': 'sub', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.366 / time elapsed:123.4 ===
[run_backtest] is executed in 124.05 seconds
{'cagr': 0.0532957817324395, 'vol': 0.2218730140993659, 'sharpe': 0.2402084901977803}
Current best sharpe
[0.2402084901977803, 0.0532957817324395, {'input1': 'assetsnc', 'input2': 'price', 'unary': 'raw', 'binary': 'sub', 'unary_out': 'raw'}]


Computing 2th factor
{'input1': 'ncfdebt', 'input2': 'ebit', 'unary': 'raw', 'binary': 'div', 'unary_out': 'rank'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.192 / time elapsed:133.5 ===
[run_backtest] is executed in 134.29 seconds
{'cagr': 0.13858652006971545, 'vol': 0.18589214298497406, 'sharpe': 0.7455211277053145}
Current best sharpe
[0.7455211277053145, 0.13858652006971545, {'input1': 'nc

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.361 / time elapsed:123.7 ===
[run_backtest] is executed in 124.42 seconds
{'cagr': 0.05545976953871358, 'vol': 0.20802460336524323, 'sharpe': 0.26660197227411137}
Current best sharpe
[0.8704397533023409, 0.1698779329180019, {'input1': 'roic', 'input2': 'fxusd', 'unary': 'zscore', 'binary': 'add', 'unary_out': 'raw'}]


Computing 17th factor
{'input1': 'epsusd', 'input2': 'roa', 'unary': 'zscore', 'binary': 'sub', 'unary_out': 'zscore'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.155 / time elapsed:95.1 ===
[run_backtest] is executed in 95.70 seconds
{'cagr': 0.13565495545361292, 'vol': 0.1984741515692835, 'sharpe': 0.6834892825137404}
Current best sharpe
[0.8704397533023409, 0.1698779329180019, {'input1': 'roic', 'input2': 'fxusd', 'unary': 'zscore', 'binary': 'add', 'unary_out': 'raw'}]


Computing 18th factor
{'input1': 

Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:1.940 / time elapsed:120.6 ===
[run_backtest] is executed in 121.33 seconds
{'cagr': 0.11642329554039181, 'vol': 0.17525633299504453, 'sharpe': 0.6643029301753322}
Current best sharpe
[0.9589077146532529, 0.17114715990209645, {'input1': 'invcapavg', 'input2': 'sgna', 'unary': 'rank', 'binary': 'sub', 'unary_out': 'raw'}]


Computing 32th factor
{'input1': 'dps', 'input2': 'roe', 'unary': 'zscore', 'binary': 'mul', 'unary_out': 'raw'}
Backtest period: 2016-01-04 00:00:00 -- 2021-08-31 00:00:00
=== date:2021-08-31 00:00:00 / total_asset:2.368 / time elapsed:96.9 ===
[run_backtest] is executed in 97.59 seconds
{'cagr': 0.1516480764796573, 'vol': 0.19830619838612049, 'sharpe': 0.7647167749360235}
Current best sharpe
[0.9589077146532529, 0.17114715990209645, {'input1': 'invcapavg', 'input2': 'sgna', 'unary': 'rank', 'binary': 'sub', 'unary_out': 'raw'}]


Computing 33th factor
{'input1': '